## link the promoter opening matrix with cell's pesudostates

In [2]:
require(data.table)
require(tidyverse)

### 1. load promoter's data 

from [summarize data at transcript level.ipynb](./summarize data at transcript level.ipynb)

In [4]:
dat.pro <- fread("gzcat ../dat/1910_v2//long_matrix_w_transcripts.txt.gz") %>% filter(!is.na(tr.idx))
dim(dat.pro)
head(dat.pro,1)
length(unique(dat.pro$cell))

[1] 18758757        7

seq,start,end,cell,value,tr.idx,gene
<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>
chr10,100027284,100028604,Islet1fresh_AGACACCTAAGAGGCAGCGTAAGAAGGCG,2,9209,LOXL4


[1] 15298

In [5]:
dat.pro <- dat.pro %>% mutate(seq = sub("chr", "", seq)) %>% mutate(prom_peak = paste0(seq, 
    ":", start, "-", end)) %>% select(prom_peak, cell) %>% distinct()
dim(dat.pro)
length(unique(dat.pro$cell))
dat.pro %>% head(1)

[1] 18758757        2

[1] 15298

prom_peak,cell
<chr>,<chr>
10:100027284-100028604,Islet1fresh_AGACACCTAAGAGGCAGCGTAAGAAGGCG


### 2. load cell's pseduostates

In [6]:
dat.ps <- do.call(rbind, lapply(c("alpha", "beta", "delta"), function(x) fread(paste0("../dat/1910_v2/", 
    x, ".pseudotime.txt"), col.names = c("cell", "ps")) %>% mutate(celltype = x)))
head(dat.ps, 1)
tail(dat.ps, 1)
dim(dat.ps)
dat.ps %>% pull(celltype) %>% table
dat.ps %>% pull(celltype) %>% table %>% sum

Warning message in fread(paste0("../dat/1910_v2/", x, ".pseudotime.txt"), col.names = c("cell", :
“Detected 1 column names but the data has 2 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”Warning message in fread(paste0("../dat/1910_v2/", x, ".pseudotime.txt"), col.names = c("cell", :
“Detected 1 column names but the data has 2 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”

cell,ps,celltype
<chr>,<dbl>,<chr>
Islet1fresh_AGACACCTAAGAGGCAAAGGAGTAGGCTC,13.98913,alpha


,cell,ps,celltype
,<chr>,<dbl>,<chr>
14526,Islet3fresh_TCCGGAGATGCAGCTACCTAGAGTGGTTGCGT,2.889196,delta


[1] 14526     3

.
alpha  beta delta 
 6218  7598   710 

[1] 14526

In [7]:
dat.pro_ps <- inner_join(dat.pro,dat.ps)
dim(dat.pro_ps)
head(dat.pro_ps,1)
length(unique(dat.pro_ps$cell))

Joining, by = "cell"


[1] 18019112        4

prom_peak,cell,ps,celltype
<chr>,<chr>,<dbl>,<chr>
10:100027284-100028604,Islet1fresh_AGACACCTAAGAGGCAGCGTAAGAAGGCG,11.8747,alpha


[1] 14526

In [8]:
dat.pro_ps%>%select(cell,celltype)%>%distinct%>%pull(celltype)%>%table

.
alpha  beta delta 
 6218  7598   710 

In [9]:
fwrite(dat.pro_ps,"../dat/1910_v2/abd.promoter.long_matrix_w_transcripts_ps.txt")

## Determine bin number 

In [3]:
dat.pro_ps<- fread("../dat/1910_v2/abd.promoter.long_matrix_w_transcripts_ps.txt")
head(dat.pro_ps,1)
dim(dat.pro_ps)

ps.range <- lapply(c("alpha", "beta", "delta"), function(ct) dat.pro_ps %>% filter(celltype == 
    ct) %>% pull(ps) %>% range)
names(ps.range) <- c("alpha", "beta", "delta")
ps.range

prom_peak,cell,ps,celltype
<chr>,<chr>,<dbl>,<chr>
10:100027284-100028604,Islet1fresh_AGACACCTAAGAGGCAGCGTAAGAAGGCG,11.8747,alpha


[1] 18019112        4

$alpha
[1]  0.00000 20.85655

$beta
[1]  0.00000 18.49255

$delta
[1]  0.00000 14.97356

In [21]:
examBinNumber <- function(N.bin = 51) {
    require('IRdisplay')
    ps.bks <- list()
    ps.bks$alpha <- seq(ps.range$alpha[1], ps.range$alpha[2] + 1e-05, length.out = N.bin)
    ps.bks$beta <- seq(ps.range$beta[1], ps.range$beta[2] + 1e-05, length.out = N.bin)
    ps.bks$delta <- seq(ps.range$delta[1], ps.range$delta[2] + 1e-05, length.out = N.bin)
    
    ps.bin.dat.sum <- rbind(dat.pro_ps %>% filter(celltype == "alpha") %>% mutate(ps_bin = cut(ps, 
        breaks = ps.bks$alpha, include.lowest = T, right = F)), dat.pro_ps %>% filter(celltype == 
        "beta") %>% mutate(ps_bin = cut(ps, breaks = ps.bks$beta, include.lowest = T, 
        right = F)), dat.pro_ps %>% filter(celltype == "delta") %>% mutate(ps_bin = cut(ps, 
        breaks = ps.bks$delta, include.lowest = T, right = F))) %>% select(-prom_peak, 
        -ps) %>% distinct()
    
    setDT(ps.bin.dat.sum)
    
    ps.bin.dat.sum <- ps.bin.dat.sum[, .(.N), by = .(ps_bin, celltype)]
    message(paste(N.bin, "# bins:\n"))
    message("quantiles of alpha cell number per bin:\n")
    t(quantile((ps.bin.dat.sum %>% filter(celltype == "alpha"))$N)) %>% display
    message("quantiles of beta cell number per bin:\n")
    t(quantile((ps.bin.dat.sum %>% filter(celltype == "beta"))$N)) %>% display
    message("quantiles of delta cell number per bin:\n")
    t(quantile((ps.bin.dat.sum %>% filter(celltype == "delta"))$N))
}

In [22]:
examBinNumber(101)

101 # bins:

quantiles of alpha cell number per bin:



0%,25%,50%,75%,100%
1,15.5,39,71.5,658


quantiles of beta cell number per bin:



0%,25%,50%,75%,100%
2,29,56,94.25,555


quantiles of delta cell number per bin:



0%,25%,50%,75%,100%
1,3,7,12,51


In [23]:
examBinNumber(51)

51 # bins:

quantiles of alpha cell number per bin:



0%,25%,50%,75%,100%
9,40.75,67.5,149.5,770


quantiles of beta cell number per bin:



0%,25%,50%,75%,100%
4,62.5,107,190.5,782


quantiles of delta cell number per bin:



0%,25%,50%,75%,100%
3,7,13,25,63


In [24]:
examBinNumber(21)

21 # bins:

quantiles of alpha cell number per bin:



0%,25%,50%,75%,100%
41,134.5,196.5,418.75,1001


quantiles of beta cell number per bin:



0%,25%,50%,75%,100%
51,231.25,335,453.25,1023


quantiles of delta cell number per bin:



0%,25%,50%,75%,100%
7,19,34.5,48.75,92
